In [ ]:
import importlib
import pandas as pd
from utils import (
    structure_noah,
    generic_helper
)
from utils.definitions import ROOT_DIR
importlib.reload(structure_noah)
importlib.reload(generic_helper)

In [ ]:
# load the structured data from notebook 01
data = generic_helper.read_data(path=ROOT_DIR, fname="data/noah_structured_data.pkl")

# load model configuration
model_config = generic_helper.load_yaml_file(
    path=f"{ROOT_DIR}/config/model_config.yaml"
)

In [ ]:
# remove cells whose end of life is less than
# the cycle at which first pulse test takes place
data = {
    k: data[k]
    for k in data
    if list(data[k]["pulses"])[0] < data[k]["summary"]["end_of_life"]
}

# dump the refined data
generic_helper.dump_data(
    data=data,
    path=f"{ROOT_DIR}/data",
    fname="noah_structured_data_refined.pkl"
)


In [ ]:
# split to train and test cells, then dump
USE_PROPOSED_SPLIT = True   # change this to False if you do not want to use the splits for generating all our results

if USE_PROPOSED_SPLIT:
    train_cells = pd.read_csv(f"{ROOT_DIR}/train_test_cells/train_cells.csv")
    test_cells = pd.read_csv(f"{ROOT_DIR}/train_test_cells/test_cells.csv")

    train_cells = {cell:data[cell] for cell in train_cells["train_cells"].values}
    test_cells = {cell:data[cell] for cell in test_cells["test_cells"].values}

else:
    cathode_groups = structure_noah.get_unique_cathode_groups(data)
    train_cells, test_cells = generic_helper.bring_splits_together(
        data=data, cathode_groups=cathode_groups, test_ratio=model_config["test_ratio"]
    )

generic_helper.dump_data(
    data={"train": train_cells, "test": test_cells},
    path=f"{ROOT_DIR}/data",
    fname="noah_train_test_split.pkl",
)

In [ ]:

# display the number of refined, train and test cells
len(data), len(train_cells), len(test_cells)
